## Conversational Interface - Chatbot with Claude-v2 and OpenSearch

### Amazon Bedrock을 사용하는 챗봇

In [33]:
installed_needed = False

In [34]:
import sys
import IPython

if installed_needed:
  print("installing deps and restarting kernel")
  !{sys.executable} -m pip install -U pip
  !{sys.executable} -m pip install -U sagemaker
  !{sys.executable} -m pip install "ipywidget>=7,<8"
  !{sys.executable} -m pip install langchain==0.0.347
  
  IPython.Application.instance().kernel.do_shutdown(True)

In [35]:
# add path

import sys, os
module_path=".."
sys.path.append(os.path.abspath(module_path))

### 1. Bedrock Client 생성

In [36]:
from utils import bedrock
from utils.bedrock import bedrock_info

boto3_bedrock = bedrock.get_bedrock_client(
  assumed_role=None,
  endpoint_url=None,
  region="us-east-1"
)

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


### 2. Titan Embedding 및 LLM Calud2-v2 모델 로딩

#### LLM loading (claude-v2)

In [37]:
from langchain.llms.bedrock import Bedrock

llm_text = Bedrock(
  client= boto3_bedrock,
  model_id="anthropic.claude-v2",
  model_kwargs={
    "max_tokens_to_sample": 512,
    "temperature": 0.1,
    "top_k": 3,
    "top_p": 0.1
  }
)
llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7fe2304bc760>, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 512, 'temperature': 0.1, 'top_k': 3, 'top_p': 0.1})

#### Embedding Model 선택

In [38]:
from langchain.embeddings.bedrock import BedrockEmbeddings

llm_emb = BedrockEmbeddings(
  client=boto3_bedrock,
  model_id="amazon.titan-embed-text-v1",
)

### 3. LangChain OpenSearch VectorStore 생성
##### opensearch-py package를 이용하여 local opensearch 연결

ref: https://opensearch.org/docs/latest/clients/python-low-level/


#### Local OpenSearch 정보

In [39]:
host = "localhost"
port = "9200"
opensearch_endpoint = f"https://{host}:{port}"
# 인증 정보
http_auth = ("admin", "admin")

# CA에 대한 ssl 정보
ca_certs_path = "root-ca.pem"

# Optional Client certificates if you don't want to use HTTP basic authentication
client_cert_path = 'admin.pem'
client_key_path = 'admin-key.pem'

#### Create client with SSL/TLS, but hostname verification disabled

In [40]:
from opensearchpy import OpenSearch

os_client = OpenSearch(
  hosts=[{'host': host, 'port': port}],
  http_compress=True,
  http_auth=http_auth,
  client_cert = client_cert_path,
  client_key = client_key_path,
  use_ssl = True,
  verify_certs = True,
  ssl_assert_hostname = False,
  ssl_show_warn=False,
  ca_certs = ca_certs_path
 )

In [41]:
# index name
index_name = "genai-demo-index-v1-with-tokenizer"

#### LangChain OpenSearch VectorStore 생성

In [42]:
from langchain.vectorstores import OpenSearchVectorSearch

vector_db = OpenSearchVectorSearch(
  index_name=index_name,
  opensearch_url=opensearch_endpoint,
  embedding_function=llm_emb,
  http_auth=http_auth,
  is_aoss=False,
  engine="faiss",
  space_type="l2",
  client_cert = client_cert_path,
  client_key = client_key_path,
  use_ssl = True,
  verify_certs = True,
  ssl_assert_hostname = False,
  ssl_show_warn=False,
  ca_certs = ca_certs_path
)

### 4. RetrievalQA 구현
##### 템플릿 생성

In [43]:
from langchain.prompts import PromptTemplate

prompt_template = """\n\nHuman: 다음 문맥의 information을 사용하여 고객 서비스 센터 직원처럼, 마지막 질문에 대한 목차 형식으로 답변을 제공하세요.

{context}

Question: {question}
\n\nAssistant:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

#### 필터 생성

In [53]:
from utils.rag import  run_RetrievalQA, show_context_used
from utils.opensearch import opensearch_utils

filter01 = "인터넷뱅킹"
filter02 = "신한은행"

query = "타기관 OTP 등록 방법 알려 주세요"

boolean_filter = opensearch_utils.get_filter(
  filter=[
    {"term": {"metadata.type": filter01}},
    {"term": {"metadata.source": filter02}}
  ]
)
print(boolean_filter)

{'bool': {'filter': [{'term': {'metadata.type': '인터넷뱅킹'}}, {'term': {'metadata.source': '신한은행'}}]}}


In [54]:
from utils import print_ww
result = run_RetrievalQA(
  llm=llm_text,
  query=query,
  prompt=PROMPT,
  vector_db=vector_db,
  boolean_filter=boolean_filter,
  k=5
)

print("#"*50)
print("query: ", query)
print("#"*50)
print_ww(result['result'])

##################################################
query:  타기관 OTP 등록 방법 알려 주세요
##################################################
 타기관 OTP 등록 방법은 다음과 같습니다:

1. 인터넷뱅킹 로그인 → 사용자관리→인터넷뱅킹관리→OTP이용등록

2. 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→ OTP이용등록

※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금한 점은 신한은행 고객센터(1599-8000)로 문의하시기 바랍니다.


### 5. Conversational Chatbot 사용한 대화형 세션

In [55]:
from langchain.chains import ConversationChain, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import BaseMessage
